## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-06-03-54-43 +0000,nyt,Judge Berates Justice Dept. in Its Prosecution...,https://www.nytimes.com/2025/11/05/us/politics...
1,2025-11-06-03-32-48 +0000,nypost,Authorities warn movie prop money circulating ...,https://nypost.com/2025/11/05/us-news/florida-...
2,2025-11-06-03-20-00 +0000,wsj,U.S. to Reduce Flight Traffic by 10% at 40 Air...,https://www.wsj.com/business/airlines/u-s-to-r...
3,2025-11-06-03-20-00 +0000,wsj,The Federal Aviation Administration said it wa...,https://www.wsj.com/business/airlines/u-s-to-r...
4,2025-11-06-03-16-13 +0000,startribune,"Takeaways: Anthony Edwards returns, but Wolves...",https://www.startribune.com/minnesota-timberwo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2322/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
103,trump,56
109,mamdani,44
228,new,29
86,election,22
25,shutdown,20


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
382,2025-11-05-06-34-00 +0000,wsj,The Democratic Party won the New York City may...,https://www.wsj.com/politics/elections/five-ta...,219
306,2025-11-05-14-02-20 +0000,nypost,Socialist NYC Mayor-elect Zohran Mamdani fires...,https://nypost.com/2025/11/05/us-news/zohran-m...,196
78,2025-11-05-23-44-26 +0000,nyt,Trump vs. Mamdani: President Sees a Capable Fo...,https://www.nytimes.com/2025/11/05/us/politics...,182
176,2025-11-05-20-26-30 +0000,nypost,New York Post’s cover on Zohran Mamdani electi...,https://nypost.com/2025/11/05/us-news/new-york...,170
182,2025-11-05-20-10-00 +0000,wsj,Zohran Mamdani sealed his mayoral victory in N...,https://www.wsj.com/politics/elections/the-vot...,167


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
382,219,2025-11-05-06-34-00 +0000,wsj,The Democratic Party won the New York City may...,https://www.wsj.com/politics/elections/five-ta...
306,130,2025-11-05-14-02-20 +0000,nypost,Socialist NYC Mayor-elect Zohran Mamdani fires...,https://nypost.com/2025/11/05/us-news/zohran-m...
293,86,2025-11-05-14-42-00 +0000,wsj,Lawmakers are hopeful they can reach a biparti...,https://www.wsj.com/politics/policy/lawmakers-...
23,59,2025-11-06-02-22-10 +0000,nypost,Erika Kirk recalls harrowing moment she found ...,https://nypost.com/2025/11/05/us-news/erika-ki...
291,55,2025-11-05-14-46-41 +0000,nyt,"Tariffs Are Here to Stay, Even if the Supreme ...",https://www.nytimes.com/2025/11/05/us/politics...
129,51,2025-11-05-22-07-00 +0000,wsj,"Maine Rep. Jared Golden, one of the most vulne...",https://www.wsj.com/politics/elections/jared-g...
158,47,2025-11-05-21-21-00 +0000,wsj,Trump urged Republican senators to jettison th...,https://www.wsj.com/politics/policy/trumps-ans...
105,44,2025-11-05-22-53-33 +0000,nypost,‘Tutors’ at Musk startup xAI had to give up ri...,https://nypost.com/2025/11/05/business/tutors-...
3,39,2025-11-06-03-20-00 +0000,wsj,The Federal Aviation Administration said it wa...,https://www.wsj.com/business/airlines/u-s-to-r...
44,37,2025-11-06-01-17-18 +0000,bbc,12 dead in crash after engine falls off cargo ...,https://www.bbc.com/news/articles/ce9dyyydjr0o...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
